In [19]:
import numpy as np
import pandas as pd 
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import seaborn as sns


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, LinearRegression, RidgeCV, LogisticRegression, LassoCV, LassoLarsIC, LogisticRegressionCV

import statsmodels.formula.api as smf
#import statsmodels.api as sm


import doubleml as dml

import matplotlib.pyplot as plt
%matplotlib inline  

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_rows", None, "display.max_columns", None)

In [22]:
#load the data
df = pd.read_csv("pension.csv")
df.shape

(9915, 44)

In [23]:
y = df[['tw']]
d = df[['p401']]
# X = df[['inc']]
X = df[["i1","i2", "i3", "i4", "i5", "i6", "i7","inc",
				"a1", "a2", "a3", "a4", "a5","fsize",
	"hs", "smcol", "col", "marr", "twoearn","db","pira","hown"]]

#### 1. OLS with controls i1-i7

In [24]:
model5 = smf.ols(formula = "y ~ d + i1+i2+ i3+ i4+i5+ i6+ i7", data = df).fit()
model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.199
Model:                            OLS   Adj. R-squared:                  0.198
Method:                 Least Squares   F-statistic:                     306.8
Date:                Sun, 29 May 2022   Prob (F-statistic):               0.00
Time:                        18:36:27   Log-Likelihood:            -1.2820e+05
No. Observations:                9915   AIC:                         2.564e+05
Df Residuals:                    9906   BIC:                         2.565e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   5.855e+04   7.06e+04      0.829      0.407   -7.99e+04    1.97e+05
d           1.503e+04   2395.740      6.274      0.000    1.03e+04    1.97e+04
i1         -4.318e+04   7.07e+04     -0.610      0.542   -1.82e+05    9.55e+04
i2          -3.87e+04   7.07e+04     -0.548      0.584   -1.77e+05    9.98e+04
i3         -2.462e+04   7.07e+04     -0.348      0.728   -1.63e+05    1.14e+05
i4         -7597.6417   7.07e+04     -0.108      0.914   -1.46e+05    1.31e+05
i5          1.028e+04   7.07e+04      0.145      0.884   -1.28e+05    1.49e+05
i6          4.034e+04   7.07e+04      0.571      0.568   -9.82e+04    1.79e+05
i7          1.361e+05   7.07e+04      1.924      0.054   -2539.717    2.75e+05
==============================================================================
Omnibus:                    10422.587   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1280370.634
Skew:                           5.064   Prob(JB):                         0.00
Kurtosis:                      57.742   Cond. No.                         222.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 2. OLS with flexible controls

I condition on all of the variables in X and their three-way interactions. This allows complex relationships between the features in the data.

In [25]:
interaction = PolynomialFeatures(degree=3, include_bias=False, interaction_only=True)
x_int = interaction.fit_transform(X)
x_intdf = pd.DataFrame(x_int, columns=interaction.get_feature_names(input_features=X.columns))
x_intdf.shape

(9915, 1793)

In [26]:
inter_df = y.join(d.join(x_intdf))
inter_df.head()

tw  p401   i1   i2   i3   i4   i5   i6   i7      inc   a1   a2   a3  \
0   53550     0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  28146.0  0.0  1.0  0.0   
1  124635     0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  32634.0  0.0  0.0  0.0   
2  192949     0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  52206.0  0.0  0.0  0.0   
3    -513     0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  45252.0  1.0  0.0  0.0   
4  212087     0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  33126.0  0.0  0.0  1.0   

    a4   a5  fsize   hs  smcol  col  marr  twoearn   db  pira  hown  i1 i2  \
0  0.0  0.0    5.0  1.0    0.0  0.0   1.0      0.0  0.0   0.0   1.0    0.0   
1  1.0  0.0    5.0  0.0    0.0  1.0   0.0      0.0  0.0   0.0   1.0    0.0   
2  1.0  0.0    3.0  0.0    0.0  0.0   1.0      1.0  0.0   1.0   1.0    0.0   
3  0.0  0.0    4.0  0.0    1.0  0.0   1.0      1.0  0.0   0.0   0.0    0.0   
4  0.0  0.0    3.0  1.0    0.0  0.0   0.0      0.0  1.0   0.0   1.0    0.0   

   i1 i3  i1 i4  i1 i5  i1 i6  i1 i7  i1 inc  i1 a1  i1 a2  i1 a3  i1 a4  \
0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0    0.0   
1    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0    0.0   
2    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0    0.0   
3    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0    0.0   
4    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0    0.0    0.0   

   i1 a5  i1 fsize  i1 hs  i1 smcol  i1 col  i1 marr  i1 twoearn  i1 db  \
0    0.0       0.0    0.0       0.0     0.0      0.0         0.0    0.0   
1    0.0       0.0    0.0       0.0     0.0      0.0         0.0    0.0   
2    0.0       0.0    0.0       0.0     0.0      0.0         0.0    0.0   
3    0.0       0.0    0.0       0.0     0.0      0.0         0.0    0.0   
4    0.0       0.0    0.0       0.0     0.0      0.0         0.0    0.0   

   i1 pira  i1 hown  i2 i3  i2 i4  i2 i5  i2 i6  i2 i7  i2 inc  i2 a1  i2 a2  \
0      0.0      0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0   
1      0.0      0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0   
2      0.0      0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0   
3      0.0      0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0   
4      0.0      0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0   

   i2 a3  i2 a4  i2 a5  i2 fsize  i2 hs  i2 smcol  i2 col  i2 marr  \
0    0.0    0.0    0.0       0.0    0.0       0.0     0.0      0.0   
1    0.0    0.0    0.0       0.0    0.0       0.0     0.0      0.0   
2    0.0    0.0    0.0       0.0    0.0       0.0     0.0      0.0   
3    0.0    0.0    0.0       0.0    0.0       0.0     0.0      0.0   
4    0.0    0.0    0.0       0.0    0.0       0.0     0.0      0.0   

   i2 twoearn  i2 db  i2 pira  i2 hown  i3 i4  i3 i5  i3 i6  i3 i7   i3 inc  \
0         0.0    0.0      0.0      0.0    0.0    0.0    0.0    0.0  28146.0   
1         0.0    0.0      0.0      0.0    0.0    0.0    0.0    0.0      0.0   
2         0.0    0.0      0.0      0.0    0.0    0.0    0.0    0.0      0.0   
3         0.0    0.0      0.0      0.0    0.0    0.0    0.0    0.0      0.0   
4         0.0    0.0      0.0      0.0    0.0    0.0    0.0    0.0      0.0   

   i3 a1  i3 a2  i3 a3  i3 a4  i3 a5  i3 fsize  i3 hs  i3 smcol  i3 col  \
0    0.0    1.0    0.0    0.0    0.0       5.0    1.0       0.0     0.0   
1    0.0    0.0    0.0    0.0    0.0       0.0    0.0       0.0     0.0   
2    0.0    0.0    0.0    0.0    0.0       0.0    0.0       0.0     0.0   
3    0.0    0.0    0.0    0.0    0.0       0.0    0.0       0.0     0.0   
4    0.0    0.0    0.0    0.0    0.0       0.0    0.0       0.0     0.0   

   i3 marr  i3 twoearn  i3 db  i3 pira  i3 hown  i4 i5  i4 i6  i4 i7   i4 inc  \
0      1.0         0.0    0.0      0.0      1.0    0.0    0.0    0.0      0.0   
1      0.0         0.0    0.0      0.0      0.0    0.0    0.0    0.0  32634.0   
2      0.0         0.0    0.0      0.0      0.0    0.0    0.0    0.0      0.0   
3      0.

In [27]:
model6 = smf.ols(formula = 'tw ~ p401 + inter_df.iloc[:,2:]', data = inter_df).fit()
model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     tw   R-squared:                       0.539
Model:                            OLS   Adj. R-squared:                  0.491
Method:                 Least Squares   F-statistic:                     11.33
Date:                Sun, 29 May 2022   Prob (F-statistic):               0.00
Time:                        18:36:37   Log-Likelihood:            -1.2546e+05
No. Observations:                9915   AIC:                         2.528e+05
Df Residuals:                    8987   BIC:                         2.595e+05
Df Model:                         927                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                   2.198e+05   9.92e+04      2.217      0.027    2.54e+04    4.14e+05
p401                        9994.9219   2046.623      4.884      0.000    5983.074     1.4e+04
inter_df.iloc[:, 2:][0]    -2.096e+05   9.81e+04     -2.137      0.033   -4.02e+05   -1.73e+04
inter_df.iloc[:, 2:][1]    -1.862e+05   9.05e+04     -2.058      0.040   -3.64e+05   -8844.807
inter_df.iloc[:, 2:][2]     -1.84e+05   8.09e+04     -2.274      0.023   -3.43e+05   -2.54e+04
inter_df.iloc[:, 2:][3]    -6.485e+04   9.69e+04     -0.669      0.503   -2.55e+05    1.25e+05
inter_df.iloc[:, 2:][4]     2.543e+05   1.65e+05      1.537      0.124   -7.01e+04    5.79e+05
inter_df.iloc[:, 2:][5]    -4.051e+05   1.62e+05     -2.504      0.012   -7.22e+05    -8.8e+04
inter_df.iloc[:, 2:][6]     1.014e+06   2.92e+05      3.475      0.001    4.42e+05    1.59e+06
inter_df.iloc[:, 2:][7]      547.5911    481.605      1.137      0.256    -396.464    1491.646
inter_df.iloc[:, 2:][8]     4.658e+04   1.09e+05      0.426      0.670   -1.68e+05    2.61e+05
inter_df.iloc[:, 2:][9]    -7.854e+04   8.12e+04     -0.968      0.333   -2.38e+05    8.06e+04
inter_df.iloc[:, 2:][10]    3.474e+04   7.56e+04      0.460      0.646   -1.13e+05    1.83e+05
inter_df.iloc[:, 2:][11]    7.459e+04   7.47e+04      0.998      0.318   -7.19e+04    2.21e+05
inter_df.iloc[:, 2:][12]    1.425e+05   9.36e+04      1.522      0.128   -4.11e+04    3.26e+05
inter_df.iloc[:, 2:][13]   -4.223e+04    2.6e+04     -1.624      0.105   -9.32e+04    8757.916
inter_df.iloc[:, 2:][14]   -6.282e+04   9.43e+04     -0.666      0.505   -2.48e+05    1.22e+05
inter_df.iloc[:, 2:][15]   -1.246e+05   9.76e+04     -1.277      0.202   -3.16e+05    6.67e+04
inter_df.iloc[:, 2:][16]   -9.583e+04   9.67e+04     -0.991      0.322   -2.85e+05    9.37e+04
inter_df.iloc[:, 2:][17]    1.235e+05   9.34e+04      1.322      0.186   -5.96e+04    3.07e+05
inter_df.iloc[:, 2:][18]   -5.893e+04    4.8e+04     -1.227      0.220   -1.53e+05    3.52e+04
inter_df.iloc[:, 2:][19]   -1.376e+05   9.52e+04     -1.445      0.148   -3.24e+05     4.9e+04
inter_df.iloc[:, 2:][20]   -2.589e+05    8.5e+04     -3.045      0.002   -4.26e+05   -9.22e+04
inter_df.iloc[:, 2:][21]   -7.117e+04   6.66e+04     -1.068      0.285   -2.02e+05    5.94e+04
inter_df.iloc[:, 2:][22]      -0.0002      0.000     -1.722      0.085      -0.000    2.83e-05
inter_df.iloc[:, 2:][23]    2.519e-05      0.000      0.234      0.815      -0.000       0.000
inter_df.iloc[:, 2:][24]       0.0006      0.000      2.588      0.010       0.000       0.001
inter_df.iloc[:, 2:][25]       0.0004      0.000      2.262      0.024    5.47e-05       0.001
inter_df.iloc[:, 2:][26]      -0.0002      0.000     -1.466      0.143      -0.000    5.41e-05
inter_df.iloc[:, 2:][27]     5.76e-05      

In [28]:

print("The coefficient estimate on d is ",np.around(model6.params['p401'],4),'. The standard error is ',np.around(model6.bse['p401'],4),'.',sep='')


The coefficient estimate on d is 9994.9219. The standard error is 2046.6234.


#### 3. Single Naive Lasso 
Here I perform an in-sample residualization of tw on the big set of controls using LassoCV, and define ytil as the in-sample residual from this estimation. In laymans terms, ytil is everything about our outcome variable that cannot be explained by all of our features in the x-matrix.

Next, I fit an ols regression of ytil on d (p401) using the statsmodel library and report the effect size estimate of p401, together with its standard error.

In [29]:
yfit = LassoCV()   
yfit.fit(x_int, y)
yhat = pd.DataFrame( yfit.predict(x_int) )
ytil = y['tw'] - yhat.iloc[:,0]

In [30]:
snl_df = d.join(pd.DataFrame(ytil,columns=['ytil']))
snl_df.head()

,p401,ytil
0,0,10637.969705
1,0,43295.529882
2,0,55566.963482
3,0,12111.818155
4,0,154677.866625


In [31]:
sm.OLS(ytil,d).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.001
Model:                            OLS   Adj. R-squared (uncentered):              0.001
Method:                 Least Squares   F-statistic:                              11.89
Date:                Sun, 29 May 2022   Prob (F-statistic):                    0.000568
Time:                        18:36:43   Log-Likelihood:                     -1.2627e+05
No. Observations:                9915   AIC:                                  2.525e+05
Df Residuals:                    9914   BIC:                                  2.525e+05
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
p401        5559.0073   1612.489      3.447      0.001    2398.201    8719.813
==============================================================================
Omnibus:                    10143.390   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1343285.301
Skew:                           4.790   Prob(JB):                         0.00
Kurtosis:                      59.211   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
model7 = smf.ols(formula = 'ytil ~ p401',data = snl_df).fit()
model7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   ytil   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     16.10
Date:                Sun, 29 May 2022   Prob (F-statistic):           6.05e-05
Time:                        18:36:43   Log-Likelihood:            -1.2626e+05
No. Observations:                9915   AIC:                         2.525e+05
Df Residuals:                    9913   BIC:                         2.525e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1969.6852    959.679     -2.052      0.040   -3850.852     -88.519
p401        7528.6925   1876.236      4.013      0.000    3850.888    1.12e+04
==============================================================================
Omnibus:                    10136.088   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1341911.913
Skew:                           4.784   Prob(JB):                         0.00
Kurtosis:                      59.184   Cond. No.                         2.46
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 4. (Manual) Double Residualization: 
I perform an in-sample residualization of y (tw) on X (big set of controls) using LassoCV from sklearn, and define ytil as the in-sample residual.

Then I perform an in-sample residualization of our treatement d (pv401) on X (big set of controls) using LogisticRegression(l2) from sklearn, and define dtil as the in-sample residual

Finally, I fit a final ols regression of ytil on dtil using the statsmodel library and report the effect size estimate of p401 from this last regression, together with its standard error.

In [33]:
model8 = LogisticRegression(penalty='l2').fit(x_int,d)
dhat = pd.DataFrame(model8.predict_proba(x_int))
d_resid = d['p401'] - dhat.iloc[:,0]

In [34]:
d_resid1 = sm.add_constant(d_resid)
model9 = sm.OLS(ytil,d_resid1).fit()
model9.params

const    2080.741751
0        4436.247324
dtype: float64

In [35]:
model9.bse

const    1112.621551
0        1591.662050
dtype: float64

In [36]:
print('The coefficient on the treatment variable from double regularization is ',np.around(np.array(model9.params)[1],4),', with a standard error of ',np.around(np.array(model9.bse)[1],4),'.',sep='')
      
      
      

The coefficient on the treatment variable from double regularization is 4436.2473, with a standard error of 1591.662.


#### 5.	Double Machine Learning with Cross Fitting

The above in-sample residualization can be improved upon using cross fitting in DML. 

The data is set up to use a Logistic/Lasso DML procedure using dml. DoubleMLData class from https://docs.doubleml.org/stable/index.html. 

I create one big dataframe with all the controls, d (p401) and y (tw), and then save the set of columns that are **controls** as a list. This dataframe, y variable, d variable, and list of controls will be passed to dml.DoubleMLData().

I use CV = 5 for Lasso and and n_folds = 3 for DML and report the effect size estimate of p401, together with its standard error.

In [40]:
#dataframe with d(p401) and y(tw)
d['y']=y

#dataframe with the controls added
df2= d.merge(inter_df.iloc[:,:],left_index=True, right_index=True).reindex()

df2.head()

p401_x       y      tw  p401_y   i1   i2   i3   i4   i5   i6   i7      inc  \
0       0   53550   53550       0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  28146.0   
1       0  124635  124635       0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  32634.0   
2       0  192949  192949       0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  52206.0   
3       0    -513    -513       0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  45252.0   
4       0  212087  212087       0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  33126.0   

    a1   a2   a3   a4   a5  fsize   hs  smcol  col  marr  twoearn   db  pira  \
0  0.0  1.0  0.0  0.0  0.0    5.0  1.0    0.0  0.0   1.0      0.0  0.0   0.0   
1  0.0  0.0  0.0  1.0  0.0    5.0  0.0    0.0  1.0   0.0      0.0  0.0   0.0   
2  0.0  0.0  0.0  1.0  0.0    3.0  0.0    0.0  0.0   1.0      1.0  0.0   1.0   
3  1.0  0.0  0.0  0.0  0.0    4.0  0.0    1.0  0.0   1.0      1.0  0.0   0.0   
4  0.0  0.0  1.0  0.0  0.0    3.0  1.0    0.0  0.0   0.0      0.0  1.0   0.0   

   hown  i1 i2  i1 i3  i1 i4  i1 i5  i1 i6  i1 i7  i1 inc  i1 a1  i1 a2  \
0   1.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0   
1   1.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0   
2   1.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0   
3   0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0   
4   1.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0    0.0    0.0   

   i1 a3  i1 a4  i1 a5  i1 fsize  i1 hs  i1 smcol  i1 col  i1 marr  \
0    0.0    0.0    0.0       0.0    0.0       0.0     0.0      0.0   
1    0.0    0.0    0.0       0.0    0.0       0.0     0.0      0.0   
2    0.0    0.0    0.0       0.0    0.0       0.0     0.0      0.0   
3    0.0    0.0    0.0       0.0    0.0       0.0     0.0      0.0   
4    0.0    0.0    0.0       0.0    0.0       0.0     0.0      0.0   

   i1 twoearn  i1 db  i1 pira  i1 hown  i2 i3  i2 i4  i2 i5  i2 i6  i2 i7  \
0         0.0    0.0      0.0      0.0    0.0    0.0    0.0    0.0    0.0   
1         0.0    0.0      0.0      0.0    0.0    0.0    0.0    0.0    0.0   
2         0.0    0.0      0.0      0.0    0.0    0.0    0.0    0.0    0.0   
3         0.0    0.0      0.0      0.0    0.0    0.0    0.0    0.0    0.0   
4         0.0    0.0      0.0      0.0    0.0    0.0    0.0    0.0    0.0   

   i2 inc  i2 a1  i2 a2  i2 a3  i2 a4  i2 a5  i2 fsize  i2 hs  i2 smcol  \
0     0.0    0.0    0.0    0.0    0.0    0.0       0.0    0.0       0.0   
1     0.0    0.0    0.0    0.0    0.0    0.0       0.0    0.0       0.0   
2     0.0    0.0    0.0    0.0    0.0    0.0       0.0    0.0       0.0   
3     0.0    0.0    0.0    0.0    0.0    0.0       0.0    0.0       0.0   
4     0.0    0.0    0.0    0.0    0.0    0.0       0.0    0.0       0.0   

   i2 col  i2 marr  i2 twoearn  i2 db  i2 pira  i2 hown  i3 i4  i3 i5  i3 i6  \
0     0.0      0.0         0.0    0.0      0.0      0.0    0.0    0.0    0.0   
1     0.0      0.0         0.0    0.0      0.0      0.0    0.0    0.0    0.0   
2     0.0      0.0         0.0    0.0      0.0      0.0    0.0    0.0    0.0   
3     0.0      0.0         0.0    0.0      0.0      0.0    0.0    0.0    0.0   
4     0.0      0.0         0.0    0.0      0.0      0.0    0.0    0.0    0.0   

   i3 i7   i3 inc  i3 a1  i3 a2  i3 a3  i3 a4  i3 a5  i3 fsize  i3 hs  \
0    0.0  28146.0    0.0    1.0    0.0    0.0    0.0       5.0    1.0   
1    0.0      0.0    0.0    0.0    0.0    0.0    0.0       0.0    0.0   
2    0.0      0.0    0.0    0.0    0.0    0.0    0.0       0.0    0.0   
3    0.0      0.0    0.0    0.0    0.0    0.0    0.0       0.0    0.0   
4    0.0      0.0    0.0    0.0    0.0    0.0    0.0       0.0    0.0   

   i3 smcol  i3 col  i3 marr  i3 twoearn  i3 db  i3 pira  i3 hown  i4 i5  \
0       0.0     0.0      1.0         0.0    0.0      0.0      1.0    0.0   
1       0.0     0.0      0.0         0.0    0.0      0.0      0.0    0.0   
2       0.0     0.0      0.0         0.0    0.0      0.0      0.0    0.0   
3       0.0     0.0      

In [38]:
controls = list(inter_df.iloc[:,2:])
controls

['i1',
 'i2',
 'i3',
 'i4',
 'i5',
 'i6',
 'i7',
 'inc',
 'a1',
 'a2',
 'a3',
 'a4',
 'a5',
 'fsize',
 'hs',
 'smcol',
 'col',
 'marr',
 'twoearn',
 'db',
 'pira',
 'hown',
 'i1 i2',
 'i1 i3',
 'i1 i4',
 'i1 i5',
 'i1 i6',
 'i1 i7',
 'i1 inc',
 'i1 a1',
 'i1 a2',
 'i1 a3',
 'i1 a4',
 'i1 a5',
 'i1 fsize',
 'i1 hs',
 'i1 smcol',
 'i1 col',
 'i1 marr',
 'i1 twoearn',
 'i1 db',
 'i1 pira',
 'i1 hown',
 'i2 i3',
 'i2 i4',
 'i2 i5',
 'i2 i6',
 'i2 i7',
 'i2 inc',
 'i2 a1',
 'i2 a2',
 'i2 a3',
 'i2 a4',
 'i2 a5',
 'i2 fsize',
 'i2 hs',
 'i2 smcol',
 'i2 col',
 'i2 marr',
 'i2 twoearn',
 'i2 db',
 'i2 pira',
 'i2 hown',
 'i3 i4',
 'i3 i5',
 'i3 i6',
 'i3 i7',
 'i3 inc',
 'i3 a1',
 'i3 a2',
 'i3 a3',
 'i3 a4',
 'i3 a5',
 'i3 fsize',
 'i3 hs',
 'i3 smcol',
 'i3 col',
 'i3 marr',
 'i3 twoearn',
 'i3 db',
 'i3 pira',
 'i3 hown',
 'i4 i5',
 'i4 i6',
 'i4 i7',
 'i4 inc',
 'i4 a1',
 'i4 a2',
 'i4 a3',
 'i4 a4',
 'i4 a5',
 'i4 fsize',
 'i4 hs',
 'i4 smcol',
 'i4 col',
 'i4 marr',
 'i4 twoearn',
 'i4 

In [37]:
# Set up basic model: Specify variables for data-backend
data_dml_base = dml.DoubleMLData(inter_df,
                                 y_col= 'tw',
                                 d_cols='p401',
                                 x_cols= controls)



In [40]:
Cs = 0.0001*np.logspace(0,4,10)
lasso = make_pipeline(StandardScaler(), LassoCV(cv=5, max_iter=1000))
logit = make_pipeline(StandardScaler(), LogisticRegressionCV(cv=5,penalty='l1', solver = 'liblinear', Cs = Cs, max_iter = 1000))

dml_model = dml.DoubleMLPLR(data_dml_base,ml_g = lasso,ml_m = logit, n_folds=3).fit(store_predictions = True)

In [46]:
print(dml_model)

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: tw
Treatment variable(s): ['p401']
Covariates: ['i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7', 'inc', 'a1', 'a2', 'a3', 'a4', 'a5', 'fsize', 'hs', 'smcol', 'col', 'marr', 'twoearn', 'db', 'pira', 'hown', 'i1 i2', 'i1 i3', 'i1 i4', 'i1 i5', 'i1 i6', 'i1 i7', 'i1 inc', 'i1 a1', 'i1 a2', 'i1 a3', 'i1 a4', 'i1 a5', 'i1 fsize', 'i1 hs', 'i1 smcol', 'i1 col', 'i1 marr', 'i1 twoearn', 'i1 db', 'i1 pira', 'i1 hown', 'i2 i3', 'i2 i4', 'i2 i5', 'i2 i6', 'i2 i7', 'i2 inc', 'i2 a1', 'i2 a2', 'i2 a3', 'i2 a4', 'i2 a5', 'i2 fsize', 'i2 hs', 'i2 smcol', 'i2 col', 'i2 marr', 'i2 twoearn', 'i2 db', 'i2 pira', 'i2 hown', 'i3 i4', 'i3 i5', 'i3 i6', 'i3 i7', 'i3 inc', 'i3 a1', 'i3 a2', 'i3 a3', 'i3 a4', 'i3 a5', 'i3 fsize', 'i3 hs', 'i3 smcol', 'i3 col', 'i3 marr', 'i3 twoearn', 'i3 db', 'i3 pira', 'i3 hown', 'i4 i5', 'i4 i6', 'i4 i7', 'i4 inc', 'i4 a1', 'i4 a2', 'i4 a3', 'i4 a4',

In [47]:
print('The estimated coefficient was 8120.89 with a standard error of 2400.3.')

The estimated coefficient was 8120.89 with a standard error of 2400.3.


### Comparing Models

**Naive OLS:**
Our first OLS with a small number of controls might be considered our 'naive' OLS. It suffers from a couple of important issues.  Firstly, we have sparse, ill conditioned data that creates issues of multicollinearity, a condition where features in our model are correlated. It is a near violation of full rank and will result in biased estimates using OLS. Even more, OLS will create unstable estimates because of its tendency to favor the specific sample its given.

**OLS with flexible controls:**
Integrating flexible controls addresses the issues of multicollinearity by accounting for the relationships between features.  The higher polynomials helps us to cover the potential of higher order relationships between the new interaction features and our outcome variable. BUT, this is still an unstable estimate and it doesn't address the fact that our treatment is likely correlated with other features.

**Naive Lasso:**
Our naive lasso helps to create more stable estimates through regularization which comes at a cost of introducing regularization bias. In addition, if features are eliminated in the LassoCV process for estimating the residuals of $tw$ that are related to d, then those eliminated features are subsumed by the error term.  This creates a relationship between our treatment, d, and the error term, which produces omitted variable bias.

**Double Residualization:** 
This method gets one step closer to the ideal model. We residualize our treatment variable, thereby removing the influence that all other features have on it, isolating the part of the treatment that can be described as its causal effect on $tw$. We have regularization bias from the residualization process, but produce more stable estimates for the the treatment effect with higher OOS performance.  Because of the orthogonalization performed in the residualization and due to the Central Limit Theorem, the 'a' and 'b' term from the DML proof goes to zero.

**DML:** 
All of the above stated for Double Residualization is true for DML, but we also utilize cross fitting in this method.  Cross fitting has the advantage of eliminating the problematic 'c' term in the DML proof. The key here is that the product of $(m-\hat{m})$ and $(g-\hat{g})$ converges to 0 faster than $\dfrac{1}{\sqrt{N}}$ .